In [1]:
import os
os.environ['BLINKA_MPR121'] = '1'
os.environ['BLINKA_FT232H'] = '1'
import time
import board
import busio
from collections import deque
import cProfile

In [4]:
i2c = busio.I2C(board.SCL, board.SDA, frequency=400000)
mpr121_address = 0x5A  # Default I2C address for the MPR121
start_reg = 0x04       # Starting register for filtered (raw) data
raw_buffer = bytearray(24)  # 24 bytes for 12 electrodes (2 bytes each)

In [5]:
HISTORY_SIZE = 100
cap_data = deque([deque(maxlen=HISTORY_SIZE) for _ in range(12)])
x_time = deque([deque(maxlen=HISTORY_SIZE) for _ in range(12)])
start_time = time.time()
with cProfile.Profile() as profiler:
    while time.time() < start_time + 1:
        while not i2c.try_lock():
            pass
        try:
            i2c.writeto_then_readfrom(mpr121_address, bytes([start_reg]), raw_buffer)
        finally:
            i2c.unlock()

        # Process the raw data for each electrode
        for chan in range(12):
            # Combine the two bytes (little-endian) for each electrode
            value = raw_buffer[2 * chan] | (raw_buffer[2 * chan + 1] << 8)
            cap_data[chan].append(value)            
            x_time[chan].append(time.time())
    profiler.print_stats(sort='tottime')
print(x_time)

         28871 function calls (26024 primitive calls) in 1.005 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      293    0.960    0.003    0.961    0.003 libusb0.py:689(__read)
      290    0.015    0.000    0.016    0.000 libusb0.py:678(__write)
 2332/583    0.003    0.000    0.007    0.000 core.py:109(wrapper)
        2    0.003    0.002    0.003    0.002 {method '__exit__' of 'sqlite3.Connection' objects}
    289/0    0.002    0.000    0.000          ftdi.py:1783(read_data_bytes)
      293    0.002    0.000    0.968    0.003 core.py:1011(read)
      583    0.002    0.000    0.002    0.000 __init__.py:517(cast)
      583    0.001    0.000    0.001    0.000 util.py:150(create_buffer)
      583    0.001    0.000    0.005    0.000 core.py:224(setup_request)
      290    0.001    0.000    0.023    0.000 core.py:978(write)
       58    0.001    0.000    0.045    0.001 i2c.py:1067(_do_read)
      290    0.001    0.000    0.0

In [8]:
print(cap_data[11])

deque([247, 247, 247, 247, 248, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 248, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 248, 247, 248, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247, 247], maxlen=100)
